## Lab 2 – Datarensning (ETL) för Smart Home IoT – “dirty version”
# Mål med labben

I denna labb ska du:

* Läsa in ett smutsigt IoT-dataset för

* smarta hem.

* Undersöka datakvalitet:

* saknade värden

* dubbletter

* orimliga värden (outliers)

* Rensa och standardisera datan    (Transform i ETL).

Slutligen Spara en ren version av datan för Lab 3 & 4 (Load).

Steg 0 – Importera bibliotek

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Steg 1 – Läs in smutsig data

För att läsa in och spara datafiler, finns det två sätt via Git eller Google driver. Förlj länken nedan där har jag förklarat hur man gör beroende på vilken väg ni väljer
https://www.notion.so/Lab2-Datarensning-2aa9de1f0d8380ab9e64e60eb26c7656?source=copy_link

In [ ]:
# läsa in den smutsiga datan, sparade från lab 1
df = pd.read_csv('../data/iot_fitness.csv')

In [ ]:
print(f"  • {200} saknade värden (NaN)")
print(f"  • {150} dubbletter")
print(f"  • {100} extrema värden (outliers)")
print(f"\n Nu ska vi lära oss hitta och fixa dessa problem!")

In [ ]:
# Visa de första raderna
df.head()

Steg 2 – Överblick av datasetet

In [ ]:
df.info()

df.describe()

Steg 3 – Kontrollera datakvalitet

4.1 Saknade värden

In [ ]:
print("Saknade värden per kolumn")
display(df.isna().sum())  # isna().sum() räknar hur många NaN finns det i varje kolumn

4.2 Dubbletter

In [ ]:
print("Antal dubbletter (hela rader som är kopior):", 
df.duplicated().sum())
dups = df[df.duplicated()]
print("Dubbletter:")
dups.head(20)  # visa de duplicerade raderna


In [ ]:
dups['maxpulse'].value_counts() # om man inte vilka ex home id som är duplicated

In [ ]:
dups['maxpulse'] 


4.3 Orimliga värden (business rules)

In [ ]:
# kollar först outliers för energyforbrukning
maxpulse = df[(df['maxpulse'] < 0) |
                     (df['maxpulse'] > 1000)]
print("Antal orimliga energivärden:", len(maxpulse))
maxpulse

In [ ]:
# kollar outliers för temp
temp_outliers = df_home_home[(df_home_home['Temp_Kök_C'] < -50) |
                   (df_home_home['Temp_Kök_C'] > 60)]
print("Antal orimliga temperaturer:", len(temp_outliers))
temp_outliners

Steg 5 – Visualisera problemet (outlier-spik)

In [ ]:
# Graf 1: Visa BARA de extrema temperaturerna (över 50°C)
extrema_temp = df_home_home[data['Temp_Kök_C'] > 50]['Temp_Kök_C']
extrema_temp.hist(color='red')
plt.title('PROBLEM: Extrema temperaturer!')
plt.xlabel('Temperatur (°C)')
plt.ylabel('Antal')
plt.show()

In [ ]:
print(" GRAF 1 VISAR:")
print(f"• {len(extrema_temp)} temperaturer vid {extrema_temp.iloc[0]:.0f}°C")
print("• Det är för varmt för ett kök! Huset skulle brinna!")

In [ ]:

# Graf 2: Visa BARA de negativa energivärdena  
negativa_energi = data[data['Vitvaror_Energi_Wh'] < 0]['Vitvaror_Energi_Wh']
negativa_energi.hist(color='orange')
plt.title('PROBLEM: Negativa energivärden!')
plt.xlabel('Energi (Wh)')  
plt.ylabel('Antal')
plt.show()

In [ ]:
print(" GRAF 2 VISAR:")
print(f"• {len(negativa_energi)} negativa energivärden vid {negativa_energi.iloc[0]:.0f} Wh")
print("• Vitvaror kan inte ge tillbaka energi - det är omöjligt!")

Steg 6 – Rensa datan

In [ ]:
# Låt oss först kolla vilka extrema värden vi har
print("🔍 EXTREMA VÄRDEN I DATAN:")
print(f"Högsta temperatur i kök: {data['Temp_Kök_C'].max():.1f}°C")
print(f"Lägsta temperatur i kök: {data['Temp_Kök_C'].min():.1f}°C")
print(f"Högsta energi: {df_home_home['Vitvaror_Energi_Wh'].max():.1f} Wh")
print(f"Lägsta energi: {df_home_home['Vitvaror_Energi_Wh'].min():.1f} Wh")

# Kolla hur många extrema värden vi har
temp_extrema = (data['Temp_Kök_C'] > 50).sum()
energi_negativ = (data['Vitvaror_Energi_Wh'] < 0).sum()
print(f"\nAntal temperaturer över 50°C: {temp_extrema}")
print(f"Antal negativa energivärden: {energi_negativ}")

6.1 – Hantera saknade värden, och NaN


In [ ]:
# skapar en kopia av datan
df_cleaned = df_home.copy()

In [ ]:
# Kolla negativa värden
negativa_energi = (data['Vitvaror_Energi_Wh'] < 0).sum()
print(f" NEGATIVA energivärden: {negativa_energi}")

In [ ]:
print(f"Före rensning: {len(df_cleaned)} rader")

# 1. Beräkna medelvärden (skippar automatiskt NaN värden)
energy_mean = df_cleaned['Vitvaror_Energi_Wh'].mean()

# 2. Fylla ALLA saknade värden (NaN, tomma celler) med medelvärden
missing_energy_before = df_cleaned['Vitvaror_Energi_Wh'].isnull().sum()
df_cleaned['Vitvaror_Energi_Wh'].fillna(energy_mean, inplace=True)

print(f"Fyllde {missing_energy_before} energi-NaN med medelvärde: {energy_mean:.2f} kWh")

In [ ]:
# Kolla saknade värden (NaN)
saknade_temp = data['Temp_Kök_C'].isnull().sum()
saknade_energi = data['Vitvaror_Energi_Wh'].isnull().sum() 
saknade_fukt = data['Fukt_Badrum_Procent'].isnull().sum()
print(f" SAKNADE värden (NaN):")
print(f"   • Temperatur: {saknade_temp}")
print(f"   • Energi: {saknade_energi}")
print(f"   • Fuktighet: {saknade_fukt}")

In [ ]:
# FYLL saknade värden med medelvärden
print("\n1️⃣ FYLLER saknade värden med medelvärden:")

# Temperatur
if data_fixed['Temp_Kök_C'].isnull().any():
    medel_temp = data_fixed['Temp_Kök_C'].mean()
    data_fixed['Temp_Kök_C'].fillna(medel_temp, inplace=True)
    print(f"   • Temperatur: Fyllde med {medel_temp:.1f}°C")

# Energi  
if data_fixed['Vitvaror_Energi_Wh'].isnull().any():
    medel_energi = data_fixed['Vitvaror_Energi_Wh'].mean()
    data_fixed['Vitvaror_Energi_Wh'].fillna(medel_energi, inplace=True)
    print(f"   • Energi: Fyllde med {medel_energi:.1f} Wh")

# Fuktighet
if data_fixed['Fukt_Badrum_Procent'].isnull().any():
    medel_fukt = data_fixed['Fukt_Badrum_Procent'].mean()
    data_fixed['Fukt_Badrum_Procent'].fillna(medel_fukt, inplace=True)
    print(f"   • Fuktighet: Fyllde med {medel_fukt:.1f}%")

6.2 Hantera dubbletter

In [ ]:
# TA BORT dubbletter
print("\n TAR BORT dubbletter:")
före_dubbletter = len(data_fixed)
data_fixed = data_fixed.drop_duplicates()
efter_dubbletter = len(data_fixed)
tagna_dubbletter = före_dubbletter - efter_dubbletter
print(f"   • Tog bort {tagna_dubbletter} dubbletter")

6.3 Hantera outliers

In [ ]:
# 3. TA BORT outliers (extrema värden)
print("\n3️⃣ TAR BORT extrema värden:")
före_outliers = len(data_fixed)
data_fixed = data_fixed[(data_fixed['Temp_Kök_C'] <= 50) & 
                        (data_fixed['Vitvaror_Energi_Wh'] >= 0)]
efter_outliers = len(data_fixed)
tagna_outliers = före_outliers - efter_outliers
print(f"   • Tog bort {tagna_outliers} extrema värden")

Steg 7 - Validera rensad data

In [ ]:
# Är rensningen OK?
print(f"\n KLART! Slutlig ren data: {len(data_fixed)} rader")
print(f" Minskade från {len(df_home)} till {len(data_fixed)} rader")


print(f" Saknade värden: {df_home.isnull().sum().sum()}")
print(f" Dubbletter: {df_home.duplicated().sum()}")
print(f" Negativ energi: {(df_home['Vitvaror_Energi_Wh'] < 0).sum()}")
print("Nu har vi perfekt data att analysera! ")

df_cleaned.head()

Steg 8 – Spara ren version

In [ ]:

# spara rensat data till csv file
df_cleaned.to_csv(
    '/content/drive/MyDrive/big_data_labs/cleaned_iot_data.csv',
    index=False
)

print("Ren version sparad som cleaned_iot_data.csv")

In [ ]:
# visa grafen efter rensning
plt.figure(figsize=(12,6))

plt.plot(df_home_home['DateTime'].head(500),
         df_home_home['Energy Consumption (kWh)'].head(500),
         label='Rådata', alpha=0.3)

plt.plot(df_home_home['DateTime'].head(500),
         df_home_home['RollingMean'].head(500),
         label='Rullande medelvärde', linewidth=2)

plt.xlabel('Tid')
plt.ylabel('kWh')
plt.title('Energiförbrukning – efter datarensning')
plt.legend()

# Fix för x-axeln:
ax = plt.gca()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))  # Max 10 etiketter
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
